In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import os
import joblib
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
import zipfile
import requests
from collections import namedtuple
from typing import List
import numpy as np
import wfdb
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.interpolate import interp1d
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader, random_split
import copy
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from mlp import *
from data import *
from ae import *
from utils import *

In [ ]:
dataset_path = download_grabmyo_dataset()

Using dataset from Google Drive: /content/drive/MyDrive/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.1.0.zip

Extracting files...
Dataset extracted to: grabmyo_data
✓ Extraction complete! Marker file created.


In [ ]:
# Configuration
DATA_ROOT = 'grabmyo_data'
GESTURE = 1
INCLUDE_CHANNELS = [9,10,11,12,13,14,15,16,18,19,20,21,22,23]
# INCLUDE_CHANNELS = [9,10,11,15]
FS = 2000
RANDOM_SEED = 42

set_random_seeds(RANDOM_SEED)
# Load raw data
metadata, signals, channel_names = load_emg_data(
    data_root=DATA_ROOT,
    gesture=GESTURE,
    include_indices=INCLUDE_CHANNELS
)

print(f"\nSignal shape: {signals.shape}")
print(f"Number of trials: {len(metadata)}")
print(f"Number of subjects: {metadata['subject'].nunique()}")
print(f"Channels: {len(channel_names)}")


# Load raw data
metadata, signals, channel_names = load_emg_data(
    data_root=DATA_ROOT,
    gesture=GESTURE,
    include_indices=INCLUDE_CHANNELS
)

print(f"\nSignal shape: {signals.shape}")
print(f"Number of trials: {len(metadata)}")
print(f"Number of subjects: {metadata['subject'].nunique()}")
print(f"Channels: {len(channel_names)}")

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

GESTURES = [1, 11, 12, 13, 16]
all_signals = []
all_metadata = []
print(f"Loading gestures: {GESTURES}...")
for g in GESTURES:
    meta, sig, _ = load_emg_data(data_root='grabmyo_data', gesture=g)
    all_signals.append(sig)
all_metadata.append(meta)
signals_raw = np.concatenate(all_signals, axis=0)
metadata_combined = pd.concat(all_metadata, ignore_index=True)
print(f"Total trials loaded: {signals_raw.shape[0]}")
signals_clean = clean_all_trials(signals_raw)

X_train, X_test = DataPreprocessor.split_data(ts_data=signals_clean)
X_train_scaled, X_test_scaled, scaler = DataPreprocessor.scale_ts_features(X_train_raw=X_train, X_test_raw=X_test)

Using device: cuda
Loading gestures: [1, 11, 12, 13, 16]...


KeyboardInterrupt: 

In [ ]:
train_tensor = torch.from_numpy(X_train_scaled).float().to(device)
test_tensor = torch.from_numpy(X_test_scaled).float().to(device)

# 1. Create the Full Training Dataset
full_train_dataset = TensorDataset(train_tensor)

# 2. Calculate Split Sizes (e.g., 85% Train, 15% Validation)
train_size = int(0.85 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size

# 3. Randomly Split
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

print(f"Training Samples: {len(train_dataset)}")
print(f"Validation Samples: {len(val_dataset)}")

# 4. Initialize Model
model = AE(
    input_len=train_tensor.shape[2],
    input_channels=1,
    latent_dim=20,
    device=device
)

X_train_split = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])
X_val_split = torch.stack([val_dataset[i][0] for i in range(len(val_dataset))])

model.train_ae(
    train_data=X_train_split,
    val_data=X_val_split,      
    model_name="emg_clean_split",
    LR=0.001,
    EPOCHS=100,
    batch_size=650,
    patience=15
)

Training Samples: 85965
Validation Samples: 15171
Starting training on cuda...
Epoch [1/100] Train Loss: 1.07713 | Val Loss: 0.99289
Model saved to emg_clean_split_best.pth
Epoch [2/100] Train Loss: 0.99645 | Val Loss: 0.98044
Model saved to emg_clean_split_best.pth
Epoch [3/100] Train Loss: 0.97528 | Val Loss: 0.95712
Model saved to emg_clean_split_best.pth
Epoch [4/100] Train Loss: 0.94003 | Val Loss: 0.91661
Model saved to emg_clean_split_best.pth
Epoch [5/100] Train Loss: 0.88912 | Val Loss: 0.86588
Model saved to emg_clean_split_best.pth
Epoch [6/100] Train Loss: 0.82367 | Val Loss: 0.80896
Model saved to emg_clean_split_best.pth
Epoch [7/100] Train Loss: 0.76572 | Val Loss: 0.75902
Model saved to emg_clean_split_best.pth
Epoch [8/100] Train Loss: 0.71603 | Val Loss: 0.71283
Model saved to emg_clean_split_best.pth
Epoch [9/100] Train Loss: 0.67045 | Val Loss: 0.68319
Model saved to emg_clean_split_best.pth
Epoch [10/100] Train Loss: 0.63479 | Val Loss: 0.65768
Model saved to emg_c

In [ ]:
# Test AE

with torch.no_grad():
    num_channels = 28
    sample_input = test_tensor[:num_channels]

    reconstruction = model(sample_input)

    mse = torch.nn.functional.mse_loss(reconstruction, sample_input)
    print(f"Test Set MSE Loss: {mse.item():.5f}")

    # Visualize
    rec_real = inverse_transform_output(reconstruction, scaler, original_num_channels=num_channels)
    print("Reconstructed Shape (Original Units):", rec_real.shape)




NameError: name 'test_tensor' is not defined

In [ ]:
# Save Scalers for later use
joblib.dump(scaler, 'ts_scaler.pkl')

NameError: name 'scaler' is not defined